# TP de régression linéaire

**Crédits.** Ce TP est largement inspiré du scénario disponible sur Wikistat.

## 1. Introduction

### Objectif

L'objectif est d'améliorer la prévision déterministe (MOCAGE), calculée par les services de Météo France, de la concentration d'ozone dans certaines stations de prélèvement. Il s'agit d'un problème dit d'adaptation statistique ou post-traitement d'une prévision locale de modèles à trop grande échelle en s'aidant d'autre variables également gérées par MétéoFrance, mais à plus petite échelle (température, force du vent, etc.).

### Description des données 
Les données ont été extraites et mises en forme par le service concerné de Météo France. Elles sont décrites par les variables suivantes:
- JOUR : Type de jour (férié (1) ou pas (0) ;
- O3obs : Concentration d'ozone effectivement observée le lendemain à 17h locales correspondant souvent au maximum de pollution observée ;
- MOCAGE : Prévision de cette pollution obtenue par un modèle déterministe de mécanique des fluides (équation de Navier et Stockes);
- TEMPE : Température prévue par MétéoFrance pour le lendemain 17h ;
- RMH2O : Rapport d'humidité ;
- NO2 : Concentration en dioxyde d'azote ;
- NO : Concentration en monoxyde d'azote ;
- STATION : Lieu de l'observation (Aix-en-Provence, Rambouillet, Munchhausen, Cadarache et Plan de Cuques) ;
- VentMOD : Force du vent ;
- VentANG : Orientation du vent.

Ce sont des données bien codées et de petite taille. Elles présentent avant tout un caractère pédagogique.


## 2. Prise en main des données

Afin de charger et d'étudier les données, nous allons utiliser la librairie pandas pour bénéficier de la classe DataFrame.


### 2.1. Chargement des données

In [ ]:
import pandas as pd
import numpy as np
from math import sqrt, log

# chargement des données
Ozone_DF = pd.read_csv('depSeuil.csv', sep = ',', header = 0)
Ozone_DF.head()


# typage des données
Ozone_DF["STATION"] = pd.Categorical(Ozone_DF["STATION"], ordered = False)
Ozone_DF["JOUR"] = pd.Categorical(Ozone_DF["JOUR"], ordered = False)
Ozone_DF["O3obs"] = pd.DataFrame(Ozone_DF["O3obs"], dtype = float)

Ozone_DF.dtypes
Ozone_DF.head()


In [ ]:

Ozone_DF.shape


### 2.2. Exploration des données

#### Exploration uni-dimensionelle

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows = 3, ncols = 3)

i_name = 0
            
for i in range(3):
    for j in range(3):
            i_name = i_name+1
            name_col = Ozone_DF.columns[i_name]
            axs[i, j].hist(Ozone_DF[name_col])
            axs[i, j].set_title(name_col)

fig.tight_layout()
plt.show()

#### Transformation du jeu de données

In [ ]:
from math import sqrt, log

Ozone_DF["SRMH2O"] = Ozone_DF["RMH2O"].map(lambda x: sqrt(x))
Ozone_DF["LNO2"] = Ozone_DF["NO2"].map(lambda x: log(x))
Ozone_DF["LNO"] = Ozone_DF["NO"].map(lambda x: log(x))
del Ozone_DF["RMH2O"]
del Ozone_DF["NO2"]
del Ozone_DF["NO"]


In [ ]:
Ozone_DF.head(5)

<H6>quelles sont les transformations appliquées ci-dessus, et dans quel but</H6><br>
On fait appel à deux fonctions de la librairie Math et on les appliques sur certains colonnes de données de DataFrame Ozone_DF comme suit:<br>
1- On fait une racine caree sur toutes les valeurs de la colonne "RMH2O" puis par la suite on affecte le resultat dans une autre nouvelle colonne crée qui s'appelle "SRMH2O".  <br>
2- On fait un logarithme (default base= 'e') sur toutes les valeurs de la colonne "N2O" puis par la suite on affecte le resultat dans une autre nouvelle colonne crée qui s'appelle "LN2O".<br>
3- On fait un logarithme (default base= 'e') sur toutes les valeurs de la colonne "NO" puis par la suite on affecte le resultat dans une autre nouvelle colonne crée qui s'appelle "LNO".<br>
4- On supprime les anciennes colonnes "RMH2O", "N2O" et "NO" du dataframe Ozone_DF.<br>
<br>En cas de problèmes, le remède consiste souvent à rechercher des transformations des variables permettant de rendre les distributions symétriques, de “banaliser” les points atypiques et de rendre linéaire la relation. Et c'est le but de ces deux transformations est celui de transformer la relation en relation linéaire. 


**Question.** Quelles sont les transformations appliquées ci-dessus, et dans quel but ?

In [ ]:
Ozone_Dum = pd.get_dummies(Ozone_DF[["JOUR", "STATION"]])
del Ozone_Dum["JOUR_0"]

Ozone_Quant = Ozone_DF[["MOCAGE", "TEMPE", "VentMOD", "VentANG", "SRMH2O", "LNO2", "LNO"]]

In [ ]:
Ozone_Dum.head(5)

In [ ]:
Ozone_Quant.head(5)

<H6>quelles sont les transformations appliquées ci-dessus, et dans quel but (2)</H6><br>
On fait appel une fonction du la librairie pandas qui est "get_dummies" on l'applique sur certains colonnes de données de DataFrame Ozone_DF comme suit:<br>
1- On fait appel a get_dummies pour convertir les variables catégorielles en variables fictives(indicateurs), et notamment sur la colonne "STATION" de Ozone_DF et aussi la colonne "JOUR" de Ozone_DF, par la suite on les stocke dans un Dataframe Ozone_Dum. <br>
2- Par la suite on supprime la colonne JOUR_0 du dataframe Ozone_Dum. <br>
3- Par la suite on cree un dataframe qui Ozone_Quan qui vas contenir tous les variables/colonnes quantitatives du Ozone_DF ("MOCAGE", "TEMPE", "VentMOD", "VentANG", "SRMH2O", "LNO2", "LNO").<br>
<br>Le but d'application de cet ensemble de transformations est de séparer les données categorielles (qu'on vas faire des etudes uniquement sur les jours feriers car on a supprimer la colonne JOUR_0 qui represente des jours normales) des données quantitatives.

**Question.** Quelles sont les transformations appliquées ci-dessus, et dans quel but ?

#### Exploration multi-dimensionnelle

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(Ozone_DF[["O3obs", "MOCAGE", "TEMPE", "VentMOD", "VentANG", "SRMH2O", "LNO2", "LNO"]], alpha=0.2, figsize=(15, 15), diagonal='kde')
plt.show()

### 2.3. Création de l'ensemble d'apprentissage / validation et de l'ensemble de test

In [ ]:
X = pd.concat([Ozone_Dum, Ozone_Quant], axis = 1)
Y = Ozone_DF["O3obs"]

X.head()


In [ ]:
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler  

X_1, X_2, Y_1, Y_2 = train_test_split(X, Y, train_size = 400, test_size = 200, random_state = 42)

X_av = X_1.to_numpy()
X_t = X_2.to_numpy()
Y_av = np.transpose([Y_1.to_numpy()])
Y_t = np.transpose([Y_2.to_numpy()])

**Remarque.** Scikit-learn fonctionne avec des numpy array. Nous allons donc utiliser les indices des colonnes:

0. JOUR
1. STATION_Aix 
2. STATION_Als 
3. STATION_Cad 
4. STATION_Pla
5. STATION_Ram
6. MOCAGE
7. TEMPE
8. VentMOD
9. VentANG
10. SRMH2O
11. LNO2
12. LNO

## 3. Régression linéaire univariée

### 3.1. Relation unidimensionelle 

On étudie la qualité de la relation $\hat{Y} = X$, où $\hat{Y}$ est le vecteur de prédiction de l'O3 observée et $X$ le vecteur des prévisions de l'O3 observée calculée par Météo France.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

fig, axs = plt.subplots(nrows = 1, ncols = 2)

axs[0].plot(X_av[:,[6]], Y_av, 'o')
axs[0].plot([0, 300], [0, 300], 'r')
axs[0].set_xlabel('Mocage: $X^6 = \hat{Y}_{av}$')
axs[0].set_ylabel('O3 observée: $Y_{av}$')

axs[1].plot(X_av[:,[6]], Y_av - X_av[:,[6]], 'o')
axs[1].plot([0, 300], [0, 0], 'r')
axs[1].set_xlabel('Mocage: $X^6= \hat{Y}_{av}$')
axs[1].set_ylabel('Résidus')

fig.tight_layout()
plt.show()

print("Coefficient de détermination :", r2_score(Y_av, X_av[:,[6]]))

<h6>Commentaires:</h6><br>
On peut dire que le coeffecient de determination R carre est tres petit avec une valeur de 0.175 approximativement, ce qui rend ce modele de Y_hat_av = X^6 (mocage) pas de tout efficace, et on vois passer aussi la droite de regression par l'origine deplan mais par sur toutes les valeurs de Y_av ou elle passe pas sur, et avec un niveau de residus tres élevé dans le deuxieme graphe(trop de valeurs "Y_hat_av - x_mocage" sont loin d'etre egale à 0, avec une difference qui arrice jusqu'à 150, ce qui fait l'augmentation des residus et aussi la baisse de R carré).

**Question.** Commentez les résultats.

### 3.2. Régression linéaire ordinaire 

On étudie la qualité de la relation $\hat{Y} = X\beta + \epsilon$, où $\hat{Y}$ est le vecteur de prédiction de l'O3 observée et $X$ le vecteur des prévisions de l'O3 observée calculée par Météo France.

In [ ]:
from sklearn import linear_model

ols = linear_model.LinearRegression()
ols.fit(X_av[:,[6]], Y_av)
Y_hat_av = ols.predict(X_av[:,[6]])

print("Coefficient de régression : \n", ols.coef_)
print()

fig, axs = plt.subplots(nrows = 1, ncols = 2, sharex = True)

axs[0].plot(X_av[:,[6]], Y_av, 'o')
axs[0].plot(X_av[:,[6]], Y_hat_av, 'r-')
#axs[0].set_ylim(0, 300)
axs[0].set_xlabel('Régression Mocage: $\hat{Y}_{av}$')
axs[0].set_ylabel('O3 observée: $Y_{av}$')

axs[1].plot(Y_hat_av, Y_av - Y_hat_av, 'o')
axs[1].plot([0, 300], [0, 0], 'r')
axs[1].set_xlabel('Prédiction Mocage: $\hat{Y}_{av}$')
axs[1].set_ylabel('Résidus')

fig.tight_layout()
plt.show()

print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, Y_hat_av))
print("Coefficient de détermination (learning set) : %.2f" % r2_score(Y_av, Y_hat_av))

<h6>Un seul coeff:</h6><br>
1- On a un seul coeffecient parceque lors du Fit de modele ols (modele de regression lineare) on utlise simplement une seule variable indepandente qui est X6_Mocage du dataframe X_av.
<br><h6>Commentaires:</h6><br>
On peut dire que le coeffecient de determination R carre est petit avec une valeur de 0.35 approximativement, ce qui rend ce modele de Y_hat_av = beta * X^6 (mocage) pas de tout efficace(tel qu'il est le cas pour Y_hat_av=x6_mocage), et on vois passer la droite de regression par certains  points (valeurs <==> que y'a certaines predictions juste) de Y_av mais pas tous, et on a moyenne de la différence au carré des valeurs réelles par rapport aux valeurs prévues (MSE) égale à 1125,92 qui comme meme trop grand. Et avec un niveau de residus tres élevé dans le deuxieme graphe(trop de valeurs residus "Y_av - Y_hat_av" sont loin d'etre egale à 0, ce qui fait l'augmentation des residus et aussi la baisse de R carré ).

**Questions.**

1. Pourquoi n'y a-t-il qu'un seul coefficient ?
2. Commentez les résultats.



## 4. Régression linéaire multivariée

### 4.1. Régression linéaire ordinaire

**Exercice.** 
1. Appliquez une régression linéaire sur l'ensemble d'apprentissage $(X_{av}, Y_{av})$ et évaluez la qualité des résultats en comparant ceux obtenus sur $(X_{av}, Y_{av})$ et sur $(X_{t}, Y_{t})$. Commentez.
2. Observez les coefficients de régression et commentez.

In [ ]:
#Application de la regression lineaire multivalué (𝑋𝑎𝑣,𝑌𝑎𝑣). 
from sklearn import linear_model

#creation de modele de regression lineaire
reg = linear_model.LinearRegression()

#entrainement de model
reg.fit(X_av, Y_av)

#affichage des coeffecients de regression pour le model
print("Coffecients de regression",reg.coef_)
print()

#prediction de model sur les deux sets X_av et X_t
Y_hat_av41 = reg.predict(X_av)
Y_hat_t41 = reg.predict(X_t)

print("****************Y_av ****************************")
print("Erreur quadratique (learning set) Y_av: %.2f" % mean_squared_error(Y_av, Y_hat_av41))
print("Coefficient de détermination (learning set) de Y_av: %.2f" % r2_score(Y_av, Y_hat_av41))
print("*************************************************")
print()

print("****************Y_t *****************************")
print("Erreur quadratique (testing set) Y_t: %.2f" % mean_squared_error(Y_t, Y_hat_t41))
print("Coefficient de détermination (testing set) de Y_t: %.2f" % r2_score(Y_t, Y_hat_t41))
print("**************************************************")

print()

#affichage des coeffecients de regression pour question 2
coef = pd.Series(reg.coef_[0], index =  X.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients pour regression lineaire multivalué")
plt.show()


<h6>Question 1:</h6><br> 
A partir des resultats obtenu, le coeffecient de determination (R carre) et la moyenne de la différence au carré des valeurs réelles par rapport aux valeurs prévues (MSE), on peut voir clairement que la qualité de couple (𝑋𝑎𝑣,𝑌𝑎𝑣) est mielleur de l'autre couple (𝑋𝑡,𝑌𝑡), ou on peu voir que le R carre de Y_hat_av41 du couple (𝑋av,𝑌av) est superieur à celui de (𝑋𝑡,𝑌𝑡), 0.57 > 0.42 qui implique que l'accuracy de model est mielleur sur le premier couple que au deuxieme. Et aussi on vois que le MSE de (𝑋𝑡,𝑌𝑡) est superieur à celui de MSE (𝑋av,𝑌av), 959.51 > 742.01 qui veut dire que le nombre de résidus est supérieur sur le couple (𝑋𝑡,𝑌𝑡) qu'au couple (𝑋av,𝑌av), ainsi éxpliquant la qualité surpassante de modele de regression sur le couple (𝑋av,𝑌av) à celui de (𝑋𝑡,𝑌𝑡).


<br><h6>Question 2:</h6><br>
On peut voir sur le plot précedent (Coefficients pour regression lineaire multivalué) que les variables les plus impactantes sur le modeles sont:<br>
on a au premier lieu, SRMHO2 est la variable indepandante la plus impactante apres on a LNO et ainsi de suite jusqu'au moins important qui est LNO2.<br>
Et on peut aussi voir que ce modele de regression il a pris tout les variables indepandantes, car elles sont toutes representés sur le plot.

### 4.2. Régression linéaire pénalisée : Ridge

**Question**.  Rappelez le modèle de la régression ridge et le comportement attendu.

Nous allons observer sur l'ensemble d'apprentissage, pour 3 valeurs de alpha ($\lambda$ dans le cours), le comportement de la regression ridge sur les coefficients.



<h6>Question</h6>RIDGE Regression model<br>
La régression ridge consiste à minimiser le critère des moindres carrés pénalisé par la norme 2 des coefficients.
<br>
Parmis les comportements attendu du Ridge on a:<br>
Si λ (alpha de Ridge) = 0, on retrouve le biais et la variance de l’estimateur des Moindres Carres Ordinaire (y'aura pas d'estimateurs Ridge).
si λ "grand" alors βˆR ≈ 0.(estimateurs de Ridge seront nulles)
Si λ est positif(augemente) ⇒ le biais augemente et la variance diminue & et réciproquement lorsque λ negatif(diminue).

#### Modèle 1

In [ ]:
r_a01 = linear_model.Ridge(alpha = 0.1)
model_r_a01 = r_a01.fit(X_av, Y_av)
Y_hat_av_r01 = r_a01.predict(X_av)
Y_hat_t_r01 = r_a01.predict(X_t)

print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, Y_hat_av_r01))
print("Coefficient de détermination (learning set) : %.2f" % r2_score(Y_av, Y_hat_av_r01))

coef = pd.Series(model_r_a01.coef_[0], index =  X.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients ridge pour alpha = 0.1")
plt.show()

#### Modèle 2

In [ ]:
r_a1 = linear_model.Ridge(alpha = 1)
model_r_a1 = r_a1.fit(X_av, Y_av)
Y_hat_av_r1 = r_a1.predict(X_av)
Y_hat_t_r1 = r_a1.predict(X_t)

print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, Y_hat_av_r1))
print("Coefficient de détermination (learning set) : %.2f" % r2_score(Y_av, Y_hat_av_r1))

coef = pd.Series(model_r_a1.coef_[0], index =  X.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients ridge pour alpha = 1")
plt.show()

#### Modèle 3

In [ ]:
r_a10 = linear_model.Ridge(alpha = 10)
model_r_a10 = r_a10.fit(X_av, Y_av)
Y_hat_av_r10 = r_a10.predict(X_av)
Y_hat_t_r10 = r_a10.predict(X_t)

print("Erreur quadratique (learning set) : %.2f" % mean_squared_error(Y_av, Y_hat_av_r10))
print("Coefficient de détermination (learning set) : %.2f" % r2_score(Y_av, Y_hat_av_r10))

coef = pd.Series(model_r_a10.coef_[0], index =  X.columns)
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients ridge pour alpha = 10")
plt.show()

**Question**. Commentez l'évolution du comportement sur les coefficients. Comment l'expliquez vous ?

**Exercice**. Visualisez globalement l'évolution des coefficients en fonction de alpha. Vous pouvez vous aider de la [documentation sur chemin de régularisation](https://scikit-learn.org/stable/auto_examples/linear_model/plot_ridge_path.html#sphx-glr-auto-examples-linear-model-plot-ridge-path-py).

**Exercice**. Mettez en place une procédure d'apprentissage rigoureuse pour trouver le paramètre alpha optimal de la régression ridge. 

**Remarque**. Pensez à regarder les références fournies en cours (cf. mail et annonce sur le moodle) ainsi que la [documentation sur le modèle Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html).

<h6>Question Evolution des comportements sur les coefficients</h6><br>
A partir des trois modeles, on peut voir qu'ils sont presque équivauts sur MSE et R carre. Concernant, les comportement des coeffecients, on peut voir sur le premier modele ou alpha = 0.1 (qui presque identique avec la regression normale qu'on a éffectué en section 4.1 sur (X_av,Y_av) comme la defintion le dis si alpha pas loin de zero alors les estimateurs de Ridge sont egals a ceux de MCO), ou on vois que le coeffecient le plus impactant est SRMH2O et que le moins impactant est LNO2; <br>
puis sur le deuxieme moedele ou alpha = 1, ou on vois que le coeffecient le plus impactant est LNO avec SRMH2O qui passe en deuxieme position et que le moins impactant est LNO2 comme le 1er modele;<br>
puis sur le troisieme moedele ou alpha = 10, ou on vois que le coeffecient le plus impactant est STATION_Pla avec LNO qui passe en deuxieme position et que le moins impactant est STATION_Aix et on vois que LNO2 monte d'un cran pour etre avant dernier;

<br>On peut expliquer çà avec ce changement de comportements des coeffecients avec la valeur de alpha, car l'estimateur Ridge est dependant du parametre alpha, par exemple si alpha tend vers 0 l'estimateur Ridge equivaut l'estimateur de MCO, et si alpha est trop grand alors estimateur de Ridge equivaut à zero.

In [ ]:
#Exercice. Visualisez globalement l'évolution des coefficients en fonction de alpha

import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model

# #############################################################################
# Compute paths

n_alphas = 200
alphas = np.logspace(5, 0, n_alphas)

coefs = []
for a in alphas:
    ridge = linear_model.Ridge(alpha=a, fit_intercept=False)
    ridge.fit(X_t, Y_t)
    coefs.append(ridge.coef_)

# #############################################################################
# Display results

ax = plt.gca()

coefs = np.array(coefs)
coefs = coefs.reshape(200,13)

ax.plot(alphas, coefs)
ax.set_xscale("log")
ax.set_xlim(ax.get_xlim()[::-1])  #  reversed axis
plt.xlabel("alpha")
plt.ylabel("Poids")
plt.title("Ridge coefficients as a function of the regularization")
plt.axis("tight")
plt.show()

In [ ]:
#Exercice. Mettez en place une procédure d'apprentissage rigoureuse pour trouver le paramètre alpha optimal de la régression ridge. 
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge

ridgeCV = RidgeCV(alphas=[0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,0.5, 1,10]).fit(X_av, Y_av)

print("Le score 'R carre' de la cross validation de Ridge Linear Regression model", ridgeCV.score(X_av, Y_av))
print("Les coeffecients du model RidgeCV Linear Regression sont:",ridgeCV.coef_)
print()
#print(ridgeCV.intercept_)
print("Le parametre alpha optimal pour la regression ridge est alpha = ",ridgeCV.alpha_)


from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score


ridge_cv=RidgeCV(alphas=[0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,0.5, 1,10], store_cv_values=True)
ridge_mod = ridge_cv.fit(X_av,Y_av)
ypred = ridge_mod.predict(X_av)

x_ax = range(len(X_av))
plt.title("Comparaison des resultats Y_hat_av avec Y avec alpha = 0.01") 
plt.scatter(x_ax,Y_av , s=10, color="blue", label="original")
plt.plot(x_ax, ypred, lw=0.8, color="red", label="predicted")
plt.legend()
plt.show()


### 4.3. Régression linéaire pénalisée : Lasso

**Question**. Rappelez le modèle de la régression lasso et le comportement attendu.

**Exercice**. Mettez en place une procédure d'apprentissage rigoureuse pour trouver le paramètre alpha optimal dans la régression lasso. Pensez à regarder la [documentation sur le modèle Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html).

<h6>Question</h6>Lasso Regression model<br>
La régression lasso consiste à minimiser le critère des moindres carrés pénalisé par la norme 1 des coefficients.<br>
Parmis les comportements attendu du Lasso on a:
<br> Si λ est positif(augemente) ⇒ le biais augemente et la variance diminue & et réciproquement lorsque λ negatif(diminue).
contrairement à ridge : λ est grand(augmente) ⇒ le nombre de coefficients nuls augmente,et c'est le cas si deux variables sont corrélées. L'une sera sélectionnée par le Lasso, l'autre supprimée. C'est aussi son avantage par rapport à une régression ridge qui ne fera pas de sélection de variables.

In [ ]:
#Exercice. Mettez en place une procédure d'apprentissage rigoureuse pour trouver le paramètre alpha optimal dans la régression lasso.


from sklearn.linear_model import LassoCV

lassocv = LassoCV(alphas=[0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,0.2,0.5, 1,10,100]).fit(X_av, Y_av)
print("Le score 'R carre' de la cross validation de Lasso Linear Regression model", lassocv.score(X_av, Y_av))
print("Les coeffecients du model LassoCV Linear Regression sont:",lassocv.coef_)
print()
print("Le parametre alpha optimal pour la regression Lasso est alpha = ",lassocv.alpha_)


from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score


lasso_cv=LassoCV(alphas=[0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,0.2,0.5, 1,10,100])
lasso_mod = lasso_cv.fit(X_av,Y_av)
ypred = lasso_mod.predict(X_av)

x_ax = range(len(X_av))
plt.title("Comparaison des resultats Y_hat_av avec Y avec alpha = 0.1") 
plt.scatter(x_ax,Y_av , s=10, color="blue", label="original")
plt.plot(x_ax, ypred, lw=0.8, color="red", label="predicted")
plt.legend()
plt.show()

## 5. Travail à rendre

Vous devez rendre sur le moodle, pour le dimanche 12/12/21, une archive nom_prenom.(g)zip ou nom_prenom.tgz inférieure à 10 Mo, contenant :

5.1. Un notebook python complétant les réponses aux questions et les exercices de ce nootebook.

5.2. Une étude sur le jeu de données [Parkinson speech](https://archive.ics.uci.edu/ml/datasets/Parkinson+Speech+Dataset+with++Multiple+Types+of+Sound+Recordings), en utilisant $Y$ = UPDRS, avec :
- Une synthèse de 4 pages au format pdf contenant les pricipales informations.
- Le notebook associé.